In [1]:
import pandas as pd
import warnings
import requests, zipfile, io, os
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
warnings.filterwarnings('ignore')

In [2]:
# download file from url, extract it and store it in the dest_folder
def download_and_extract(url, dest_folder):
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall(dest_folder)

In [3]:
# define folders and urls
healthy_path = './healthy'
healthy_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewHealthy/Signal.zip'
parkinson_path = './parkinson'
parkinson_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewPatients/Signal.zip'

In [4]:
download_and_extract(healthy_url, healthy_path)

In [5]:
download_and_extract(parkinson_url, parkinson_path)

In [6]:
# actual downloaded signals
healthy_signals = f'{healthy_path}/Signal/'
parkinson_signals = f'{parkinson_path}/Signal/'

In [7]:
# define Input and labels (0 = healthy, parkinson = 1)
X = []
y = []
seq = 500

In [8]:
# create dataset with sequence length 500

# healthy signals
for filename in os.listdir(healthy_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(healthy_signals + filename, sep='\s+', header=None, comment='#')
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(0)

# parkinson signals
for filename in os.listdir(parkinson_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(parkinson_signals + filename, sep='\s+', header=None, comment='#')
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(1) 


In [9]:
len(X)

29088

In [10]:
# convert lists to np arrays
X = np.array(X)
y = np.array(y)

In [11]:
X.shape

(29088, 500, 6)

In [12]:
y.shape

(29088,)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

In [14]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_valid = scaler.transform(X_valid.reshape(-1, X_valid.shape[-1])).reshape(X_valid.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [15]:
model = keras.models.Sequential([
keras.layers.LSTM(128, return_sequences=True,  input_shape=(X_train.shape[1], 6)),
keras.layers.LSTM(128),
keras.layers.Dense(1, activation="sigmoid")
])


In [16]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [17]:
checkpoint_filepath = "model.h5"
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [18]:
model.fit(X_train, y_train, epochs = 30, batch_size = 32, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

Epoch 1/30
582/582 [==============================] - 30s 52ms/step - loss: 0.6103 - accuracy: 0.6753 - val_loss: 0.6237 - val_accuracy: 0.6764
Epoch 2/30
582/582 [==============================] - 30s 51ms/step - loss: 0.5935 - accuracy: 0.6846 - val_loss: 0.6061 - val_accuracy: 0.6848
Epoch 3/30
582/582 [==============================] - 30s 51ms/step - loss: 0.5853 - accuracy: 0.6932 - val_loss: 0.5822 - val_accuracy: 0.6960
Epoch 4/30
582/582 [==============================] - 30s 51ms/step - loss: 0.6026 - accuracy: 0.6781 - val_loss: 0.6451 - val_accuracy: 0.6253
Epoch 5/30
582/582 [==============================] - 30s 51ms/step - loss: 0.6120 - accuracy: 0.6751 - val_loss: 0.6119 - val_accuracy: 0.6367
Epoch 6/30
582/582 [==============================] - 30s 51ms/step - loss: 0.6209 - accuracy: 0.6636 - val_loss: 0.6178 - val_accuracy: 0.6758
Epoch 7/30
582/582 [==============================] - 30s 51ms/step - loss: 0.6068 - accuracy: 0.6769 - val_loss: 0.5915 - val_accuracy:

In [19]:
model.fit(X_train, y_train, epochs = 20, batch_size = 32, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

Epoch 1/20
582/582 [==============================] - 30s 51ms/step - loss: 0.4394 - accuracy: 0.7987 - val_loss: 0.4321 - val_accuracy: 0.8053
Epoch 2/20
582/582 [==============================] - 30s 51ms/step - loss: 0.4176 - accuracy: 0.8096 - val_loss: 0.4289 - val_accuracy: 0.8000
Epoch 3/20
582/582 [==============================] - 30s 51ms/step - loss: 0.4049 - accuracy: 0.8169 - val_loss: 0.4116 - val_accuracy: 0.8161
Epoch 4/20
582/582 [==============================] - 30s 51ms/step - loss: 0.4140 - accuracy: 0.8118 - val_loss: 0.4480 - val_accuracy: 0.7948
Epoch 5/20
582/582 [==============================] - 30s 51ms/step - loss: 0.4073 - accuracy: 0.8141 - val_loss: 0.3925 - val_accuracy: 0.8180
Epoch 6/20
582/582 [==============================] - 30s 51ms/step - loss: 0.3909 - accuracy: 0.8224 - val_loss: 0.3890 - val_accuracy: 0.8223
Epoch 7/20
582/582 [==============================] - 30s 51ms/step - loss: 0.3719 - accuracy: 0.8323 - val_loss: 0.3828 - val_accuracy:

In [20]:
model.fit(X_train, y_train, epochs = 20, batch_size = 32, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

Epoch 1/20
582/582 [==============================] - 30s 51ms/step - loss: 0.2693 - accuracy: 0.8850 - val_loss: 0.3294 - val_accuracy: 0.8558
Epoch 2/20
582/582 [==============================] - 30s 51ms/step - loss: 0.2589 - accuracy: 0.8888 - val_loss: 0.2919 - val_accuracy: 0.8792
Epoch 3/20
582/582 [==============================] - 30s 51ms/step - loss: 0.2567 - accuracy: 0.8903 - val_loss: 0.3037 - val_accuracy: 0.8670
Epoch 4/20
582/582 [==============================] - 30s 51ms/step - loss: 0.2477 - accuracy: 0.8955 - val_loss: 0.3058 - val_accuracy: 0.8672
Epoch 5/20
582/582 [==============================] - 30s 51ms/step - loss: 0.2407 - accuracy: 0.8990 - val_loss: 0.2883 - val_accuracy: 0.8749
Epoch 6/20
582/582 [==============================] - 30s 51ms/step - loss: 0.2286 - accuracy: 0.9051 - val_loss: 0.2943 - val_accuracy: 0.8741
Epoch 7/20
582/582 [==============================] - 30s 51ms/step - loss: 0.2247 - accuracy: 0.9087 - val_loss: 0.3075 - val_accuracy:

In [21]:
model.load_weights(checkpoint_filepath)

In [22]:
model.evaluate(X_valid, y_valid)


146/146 [==============================] - 3s 17ms/step - loss: 0.2772 - accuracy: 0.8919


[0.27720537781715393, 0.8919209241867065]

In [23]:
model.evaluate(X_test, y_test)

182/182 [==============================] - 3s 17ms/step - loss: 0.2805 - accuracy: 0.8909


[0.28049570322036743, 0.8908559679985046]

In [24]:
!ls

healthy  model.h5  parkinson  sample_data


In [25]:
from google.colab import files
files.download('model.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>